# **Cuaderno: 2. Procesamiento de Bronce a Plata**

### **Carga de Datos desde la Capa Bronce**

**Objetivo:** El primer paso en este cuaderno de transformación es cargar los datos que ya hemos ingerido. Esta celda se conecta a nuestra base de datos, lee las tablas de la capa Bronce y las carga en DataFrames de PySpark, preparándolas para el proceso de limpieza y refinamiento.

-----

## **Preparando el Entorno de Transformación**

Antes de poder aplicar cualquier lógica de negocio, necesitamos tener acceso a los datos. En lugar de generar los datos nuevamente, nos conectamos directamente a las tablas Delta que creamos en el cuaderno anterior.

Este proceso simula cómo un job o tarea de ETL (Extracción, Transformación y Carga) comenzaría su ejecución:

1.  **Establecer el Contexto de la Base de Datos**: Indicamos a Spark que todas las operaciones subsecuentes deben realizarse dentro de nuestra base de datos `curso_arquitecturas`.
2.  **Cargar las Tablas en DataFrames**: Leemos cada tabla de la capa Bronce y la asignamos a un DataFrame de PySpark. Nombrar las variables con un sufijo (ej. `_df`) es una buena práctica para identificar fácilmente que se trata de un DataFrame en memoria.
3.  **Verificación Inicial**: Realizamos una acción simple como `.count()` para verificar que los datos se han cargado correctamente y para tener una idea del volumen de registros con el que vamos a trabajar.

-----

In [0]:

# Celda 1: Configuración del entorno y carga de tablas de la capa Bronce.

# Paso 1: Establecer la base de datos actual para la sesión de Spark.
# Esto nos evita tener que escribir el nombre de la base de datos en cada consulta.
db_name = "curso_arquitecturas"
spark.sql(f"USE {db_name}")

print(f"Contexto establecido en la base de datos: '{db_name}'")

# Paso 2: Leer cada tabla de la capa Bronce y cargarla en un DataFrame.
try:
    proveedores_bronze_df = spark.read.table("proveedores_bronze")
    usuarios_bronze_df = spark.read.table("usuarios_bronze")
    productos_bronze_df = spark.read.table("productos_bronze")
    pedidos_bronze_df = spark.read.table("pedidos_bronze")
    devoluciones_bronze_df = spark.read.table("devoluciones_bronze")
    
    print("\nCarga de datos desde la capa Bronce completada exitosamente.")

except Exception as e:
    print(f"Error al cargar las tablas de la capa Bronce: {e}")
    # En un entorno de producción, aquí podrías detener la ejecución o enviar una alerta.
    # dbutils.notebook.exit("Error crítico: No se pudieron cargar las tablas de origen.")

# Paso 3: Realizar una verificación rápida para confirmar que los DataFrames tienen datos.
print("\n--- Conteo de registros por tabla ---")
print(f"Proveedores: {proveedores_bronze_df.count()} registros")
print(f"Usuarios: {usuarios_bronze_df.count()} registros")
print(f"Productos: {productos_bronze_df.count()} registros")
print(f"Pedidos: {pedidos_bronze_df.count()} registros")
print(f"Devoluciones: {devoluciones_bronze_df.count()} registros")

Pasar de la capa **Bronce** a la **Plata** es el proceso de transformar datos crudos y poco fiables en un conjunto de datos limpio, estructurado y listo para el análisis.

Es el paso donde se aplica la calidad y se asegura que la información sea confiable.

---
## **El Proceso de Refinamiento**

El objetivo es convertir datos en su estado original (Bronce) a un formato validado y útil (Plata) mediante estas acciones clave:

* **Limpieza:** Se eliminan registros duplicados, se corrigen errores evidentes y se manejan los valores nulos.
* **Validación:** Se asegura que cada columna tenga el tipo de dato correcto (ej. convertir texto a fecha o número) y que los identificadores importantes no estén vacíos.
* **Estructuración:** Se aplican nombres de columna claros y consistentes y se organizan los datos en un modelo lógico y fácil de consultar.

En resumen, la capa Plata toma el desorden de la capa Bronce y lo convierte en la **fuente única de la verdad** para la empresa.

# Justificación y Plan para el Modelo Dimensional (Kimball)**

**Objetivo:** Antes de escribir el código de transformación, esta celda establece la estrategia que seguiremos. Adoptaremos formalmente la metodología de **modelado dimensional de Kimball** para estructurar nuestras tablas en la capa Plata.

---
## **¿Por Qué un Modelo Dimensional?**

La estructura de los datos en la capa Plata no es arbitraria. Debe diseñarse específicamente para optimizar las consultas analíticas y facilitar la comprensión del negocio. El modelo Kimball es el estándar de la industria para este propósito por tres razones principales:

1.  **Rendimiento en Consultas**: A diferencia de los modelos normalizados para sistemas transaccionales (que optimizan la escritura), el modelo dimensional (esquema en estrella) está diseñado para acelerar las operaciones de lectura y agregación, que son el 99% de las cargas de trabajo en un Data Warehouse.

2.  **Claridad para el Análisis**: Separa los datos de forma intuitiva en **"Hechos"** (los números y métricas que queremos medir, como las ventas) y **"Dimensiones"** (el contexto que describe esos hechos: quién, qué, dónde, cuándo). Esta estructura es fácil de entender para los analistas y herramientas de BI.

3.  **Estándar de la Industria**: Es una metodología probada, robusta y ampliamente adoptada, lo que facilita la mantenibilidad del proyecto y la incorporación de nuevos miembros al equipo.

---
## **Nuestro Plan de Acción**

Para implementar el modelo, clasificaremos nuestras tablas `_silver` y crearemos una dimensión adicional clave:

* **Identificar Tablas de Dimensión**: Estas tablas describen nuestras entidades de negocio. Serán:
    * `usuarios_silver` (Dimensión de Cliente)
    * `productos_silver` (Dimensión de Producto)
    * `proveedores_silver` (Dimensión de Proveedor)

* **Identificar Tablas de Hechos**: Estas tablas registran los eventos de negocio. Serán:
    * `pedidos_silver` (Hechos de Ventas)
    * `devoluciones_silver` (Hechos de Devoluciones)

* **Crear una Dimensión de Tiempo (Mejor Práctica)**: Una de las prácticas fundamentales del modelo Kimball es no usar directamente las columnas de fecha (`TIMESTAMP`) de las tablas de hechos para el análisis. En su lugar, crearemos una **Dimensión de Fecha** (`dim_fecha`). Esta tabla contendrá una fila por cada día y columnas precalculadas (año, mes, nombre del mes, trimestre, día de la semana, etc.), lo que simplificará y acelerará enormemente las consultas basadas en tiempo.

Al finalizar las siguientes celdas, nuestra capa Plata estará organizada como un esquema en estrella, listo para servir análisis o para construir las agregaciones finales en la capa Oro.

![ERD](ERD.png)

## Intermezzo

El flujo de diseño siempre sigue esta secuencia: **Conceptual -> Lógico -> Físico**.

---
## 1. Modelo Conceptual

**¿Qué es?** Es la vista de más alto nivel. Identifica las principales **entidades de negocio** y las relaciones entre ellas, sin entrar en detalles de atributos o llaves. Responde a la pregunta: ¿de qué se trata este negocio?

**¿Cuándo lo hicimos en el taller?**
Lo hicimos al principio, en la **primera celda**, cuando decidimos qué datos necesitábamos generar para que nuestro escenario fuera realista. En ese momento, identificamos los conceptos clave:
* Hay **Usuarios** que realizan **Pedidos**.
* Los **Pedidos** contienen **Productos**.
* Los **Productos** son suministrados por **Proveedores**.
* A veces, los **Pedidos** resultan en **Devoluciones**.

Esa discusión inicial y la planificación de las entidades a generar fue nuestro modelo conceptual.


---
## 2. Modelo Lógico

**¿Qué es?** Es el plano detallado de nuestra base de datos, pero sin atarse a una tecnología específica. Aquí definimos las tablas, las columnas (atributos), los tipos de datos generales (número, texto, fecha), y las **claves primarias y foráneas** que conectan las tablas.

**¿Cuándo lo hicimos en el taller?**
Lo construimos en la fase de justificación, justo antes de empezar a escribir el código para la capa Plata:
* Cuando decidimos usar la **metodología Kimball** y su **esquema en estrella**.
* Cuando diseñamos el **Diagrama Entidad-Relación (ERD) en Mermaid**. Ese diagrama es la representación visual de nuestro modelo lógico. Define con precisión cada tabla, columna y relación.

*es decir aqui arribita*

---
## 3. Modelo Físico

**¿Qué es?** Es la implementación real y tecnológica del modelo lógico en un sistema de base de datos específico. Aquí es donde se toman decisiones sobre el motor de almacenamiento, los tipos de datos exactos, los índices y las optimizaciones físicas.


-----
Ya que conocemos el ERD y lo que buscamos construir, vamos a crear las tablas en la capa Plata con las llaves apropiadas. Para ello, seguiremos un proceso de tres pasos:

1.  Crear las Tablas de Dimensión y declarar sus claves primarias.
2.  Construir la Dimensión de Fecha, una pieza clave en nuestro modelo.
3.  Crear las Tablas de Hechos y, lo más importante, establecer formalmente las relaciones con las dimensiones mediante claves foráneas.

Comencemos.

-----

### **Paso 1: Creación de las Dimensiones y Declaración de sus Claves Primarias**

**Objetivo:** Crear las tablas de dimensión y, acto seguido, declarar sus claves primarias (PK).

**Código:**


In [0]:
%sql
USE curso_arquitecturas;

-- **Dimensión de Usuarios**
-- Paso 1: Crear la tabla con el esquema definido, especificando NOT NULL en la PK.
CREATE OR REPLACE TABLE usuarios_silver (
  id_usuario BIGINT NOT NULL,
  nombre_usuario STRING,
  email STRING,
  fecha_registro TIMESTAMP,
  ciudad STRING
) COMMENT 'Dimensión de clientes con PK definida como NOT NULL.';

-- Paso 2: Insertar los datos en la tabla ya creada.
INSERT INTO usuarios_silver
SELECT CAST(id_usuario AS BIGINT), nombre, email, CAST(fecha_registro AS TIMESTAMP), ciudad
FROM usuarios_bronze WHERE id_usuario IS NOT NULL;

-- Paso 3: Ahora la declaración de la PK funcionará.
ALTER TABLE usuarios_silver ADD CONSTRAINT pk_usuarios PRIMARY KEY(id_usuario) NOT ENFORCED;


-- **Dimensión de Proveedores**
CREATE OR REPLACE TABLE proveedores_silver (
  id_proveedor BIGINT NOT NULL,
  nombre_proveedor STRING
);
INSERT INTO proveedores_silver
SELECT CAST(id_proveedor AS BIGINT), nombre_proveedor
FROM proveedores_bronze WHERE id_proveedor IS NOT NULL;
ALTER TABLE proveedores_silver ADD CONSTRAINT pk_proveedores PRIMARY KEY(id_proveedor) NOT ENFORCED;


-- **Dimensión de Productos**
CREATE OR REPLACE TABLE productos_silver (
  id_producto BIGINT NOT NULL,
  id_proveedor BIGINT,
  nombre_producto STRING,
  categoria STRING,
  precio_unitario DECIMAL(10, 2)
);
INSERT INTO productos_silver
SELECT CAST(id_producto AS BIGINT), CAST(id_proveedor AS BIGINT), nombre_producto, categoria, CAST(precio_unitario AS DECIMAL(10, 2))
FROM productos_bronze WHERE id_producto IS NOT NULL;
ALTER TABLE productos_silver ADD CONSTRAINT pk_productos PRIMARY KEY(id_producto) NOT ENFORCED;

#### **¿Por qué declaramos estas Claves Primarias?**

Aunque son `NOT ENFORCED` (no forzadas), al declarar una Clave Primaria le estamos dando al optimizador de consultas una garantía: **los valores en esta columna son únicos**. Con esta información, el motor puede simplificar los planes de consulta, por ejemplo, al evitar pasos de eliminación de duplicados después de un `JOIN`, sabiendo de antemano que la relación es de uno a muchos.

-----

### **Paso 2: Construcción de la Dimensión de Fecha**

**Objetivo:** Construir la `dim_fecha` y declarar su clave primaria, un requisito para poder vincularla de manera eficiente a las tablas de hechos.

**Código:**

In [0]:
%sql
-- Paso 1: Crear la tabla vacía con el esquema definido, especificando NOT NULL en la PK.
CREATE OR REPLACE TABLE dim_fecha (
  id_fecha DATE NOT NULL,
  anio INT,
  mes INT,
  dia INT,
  trimestre INT,
  nombre_mes STRING,
  nombre_dia_semana STRING,
  tipo_dia STRING
) COMMENT 'Dimensión de fecha con PK definida como NOT NULL.';

-- Paso 2: Insertar los datos en la tabla ya creada.
INSERT INTO dim_fecha
SELECT
  fecha AS id_fecha,
  YEAR(fecha) AS anio,
  MONTH(fecha) AS mes,
  DAY(fecha) AS dia,
  QUARTER(fecha) AS trimestre,
  DATE_FORMAT(fecha, 'MMMM') AS nombre_mes, 
  DATE_FORMAT(fecha, 'EEEE') AS nombre_dia_semana,
  CASE WHEN DAYOFWEEK(fecha) IN (1, 7) THEN 'Fin de Semana' ELSE 'Día de Semana' END AS tipo_dia
FROM (
  SELECT EXPLODE(SEQUENCE(TO_DATE('2022-01-01'), TO_DATE('2026-12-31'), INTERVAL 1 DAY)) AS fecha
);

-- Paso 3: Ahora la declaración de la PK funcionará.
ALTER TABLE dim_fecha ADD CONSTRAINT pk_dim_fecha PRIMARY KEY(id_fecha) NOT ENFORCED;

-----

### **Paso 3: Creación de Tablas de Hechos y sus Claves Foráneas**

**Objetivo:** Construir las tablas de hechos y declarar las relaciones con sus dimensiones a través de las claves foráneas (FK). Este es el paso final para completar nuestro esquema en estrella.

**Código:**

In [0]:
%sql
-- **Tabla de Hechos de Pedidos**
-- Paso 1: Crear la tabla vacía con el esquema definido, especificando NOT NULL en la PK.
CREATE OR REPLACE TABLE pedidos_silver (
  id_pedido BIGINT NOT NULL,
  id_usuario BIGINT,
  id_producto BIGINT,
  id_fecha DATE,
  cantidad INT,
  monto_total DECIMAL(18, 2),
  ts_pedido TIMESTAMP
);

-- Paso 2: Insertar los datos en la tabla ya creada.
INSERT INTO pedidos_silver
SELECT 
  CAST(p.id_pedido AS BIGINT), 
  CAST(p.id_usuario AS BIGINT), 
  CAST(p.id_producto AS BIGINT), 
  TO_DATE(p.fecha_pedido), 
  CAST(p.cantidad AS INT), 
  CAST(p.monto AS DECIMAL(18, 2)), 
  CAST(p.fecha_pedido AS TIMESTAMP)
FROM pedidos_bronze AS p 
WHERE p.id_pedido IS NOT NULL AND p.id_usuario IS NOT NULL AND p.id_producto IS NOT NULL;

-- Paso 3: Ahora las declaraciones de llaves funcionarán.
ALTER TABLE pedidos_silver ADD CONSTRAINT pk_pedidos PRIMARY KEY(id_pedido) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_usuarios FOREIGN KEY(id_usuario) REFERENCES usuarios_silver(id_usuario) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_productos FOREIGN KEY(id_producto) REFERENCES productos_silver(id_producto) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_fecha FOREIGN KEY(id_fecha) REFERENCES dim_fecha(id_fecha) NOT ENFORCED;


-- **Tabla de Hechos de Devoluciones**
CREATE OR REPLACE TABLE devoluciones_silver (
  id_devolucion BIGINT NOT NULL,
  id_pedido BIGINT,
  id_fecha DATE,
  motivo STRING,
  ts_devolucion TIMESTAMP
);
INSERT INTO devoluciones_silver
SELECT 
  CAST(d.id_devolucion AS BIGINT), 
  CAST(d.id_pedido AS BIGINT), 
  TO_DATE(d.fecha_devolucion), 
  d.motivo, 
  CAST(d.fecha_devolucion AS TIMESTAMP)
FROM devoluciones_bronze AS d 
WHERE d.id_devolucion IS NOT NULL AND d.id_pedido IS NOT NULL;

-- Declaración de Restricciones
ALTER TABLE devoluciones_silver ADD CONSTRAINT pk_devoluciones PRIMARY KEY(id_devolucion) NOT ENFORCED;
ALTER TABLE devoluciones_silver ADD CONSTRAINT fk_devoluciones_pedidos FOREIGN KEY(id_pedido) REFERENCES pedidos_silver(id_pedido) NOT ENFORCED;
ALTER TABLE devoluciones_silver ADD CONSTRAINT fk_devoluciones_fecha FOREIGN KEY(id_fecha) REFERENCES dim_fecha(id_fecha) NOT ENFORCED;


#### **¿Por qué declaramos estas Claves Foráneas?**

Declarar las claves foráneas es el paso de optimización más crítico. Al hacerlo, le damos al optimizador un "mapa" de cómo se conectan las tablas. Esto permite dos optimizaciones clave:

1.  **Reordenamiento de `JOINs`**: El optimizador puede elegir el orden más eficiente para unir las tablas, empezando por las uniones que más reduzcan los datos a procesar.
2.  **Eliminación de `JOINs`**: Si una consulta une `pedidos_silver` con `usuarios_silver` solo para filtrar por `id_usuario`, el optimizador, al ver la relación FK-PK, entiende que el `JOIN` es innecesario y lo elimina, leyendo únicamente la tabla de hechos. Esto acelera drásticamente la consulta.


### **Implementación del Modelo Físico Optimizado**

Ahora vamos a transformar nuestro modelo dimensional, que ya es estructuralmente correcto, en un modelo de alto rendimiento. Para ello, aplicaremos técnicas de optimización física que organizan la disposición de los datos, permitiendo a Databricks leer la menor cantidad de información posible para resolver una consulta.

#### **La Estrategia: Habilitar el "Data Skipping"**

Un modelo sin optimización física obliga a Databricks a realizar un escaneo completo de todos los archivos de datos para la mayoría de las consultas, lo cual es ineficiente y costoso a gran escala.

Para evitarlo, aplicaremos las dos técnicas de optimización física más importantes en Delta Lake para habilitar el **"data skipping"** (omisión de datos):

1.  **Particionamiento (`PARTITIONED BY`)**: Esta técnica divide una tabla en subdirectorios basados en los valores de una columna. Cuando una consulta filtra por esa columna, Databricks solo lee los directorios relevantes, ignorando el resto. Es la optimización más efectiva para columnas de **baja cardinalidad** (pocos valores distintos), como fechas, países o categorías.

2.  **Z-Ordering (`ZORDER BY`)**: Esta es una técnica más fina que reorganiza los datos *dentro* de los archivos. Agrupa valores de columnas relacionadas para que estén físicamente cerca. Esto mejora drásticamente la capacidad del motor para "saltarse" bloques de datos dentro de un archivo, incluso sin un filtro de partición. Es ideal para columnas de **alta cardinalidad** que se usan frecuentemente en filtros o `JOINs`, como las claves primarias y foráneas (`id_usuario`, `id_producto`).

A continuación, reconstruiremos nuestras tablas de la capa Plata aplicando estas técnicas.

-----

### **Creación de Dimensiones Optimizadas**

**Objetivo:** Crear las tablas de dimensión, aplicando particionamiento y Z-Ordering para acelerar las futuras consultas y uniones.

**Código:**


In [0]:
%sql
USE curso_arquitecturas;

-- **Dimensión de Usuarios (Optimizada con Z-Ordering)**
CREATE OR REPLACE TABLE usuarios_silver (
  id_usuario BIGINT NOT NULL, nombre_usuario STRING, email STRING, fecha_registro TIMESTAMP, ciudad STRING
) COMMENT 'Dimensión de clientes optimizada con Z-Ordering en su PK.';
INSERT INTO usuarios_silver
SELECT CAST(id_usuario AS BIGINT), nombre, email, CAST(fecha_registro AS TIMESTAMP), ciudad
FROM usuarios_bronze WHERE id_usuario IS NOT NULL;
OPTIMIZE usuarios_silver ZORDER BY (id_usuario);
ALTER TABLE usuarios_silver ADD CONSTRAINT pk_usuarios PRIMARY KEY(id_usuario) NOT ENFORCED;

-- **Dimensión de Productos (Optimizada con Particionamiento y Z-Ordering)**
CREATE OR REPLACE TABLE productos_silver (
  id_producto BIGINT NOT NULL, id_proveedor BIGINT, nombre_producto STRING, precio_unitario DECIMAL(10, 2), categoria STRING
)
PARTITIONED BY (categoria)
COMMENT 'Dimensión de productos particionada por categoría y optimizada por su PK.';
INSERT INTO productos_silver
SELECT CAST(id_producto AS BIGINT), CAST(id_proveedor AS BIGINT), nombre_producto, CAST(precio_unitario AS DECIMAL(10, 2)), categoria
FROM productos_bronze WHERE id_producto IS NOT NULL;
OPTIMIZE productos_silver ZORDER BY (id_producto);
ALTER TABLE productos_silver ADD CONSTRAINT pk_productos PRIMARY KEY(id_producto) NOT ENFORCED;


* **Explicación de la Optimización:** Se particiona `productos_silver` por `categoria` porque es una columna de baja cardinalidad, lo que acelerará los filtros por este campo. Se aplica `ZORDER BY` en las claves primarias (`id_usuario`, `id_producto`) porque son columnas de alta cardinalidad usadas en los `JOINs`.


### Creación de la Dimensión de Fecha**

*(Esta tabla generalmente no requiere optimizaciones adicionales debido a su naturaleza secuencial, por lo que el código no cambia)*

In [0]:
%sql
CREATE OR REPLACE TABLE dim_fecha (id_fecha DATE NOT NULL, anio INT, mes INT, dia INT, trimestre INT, nombre_mes STRING, nombre_dia_semana STRING, tipo_dia STRING);
INSERT INTO dim_fecha SELECT fecha, YEAR(fecha), MONTH(fecha), DAY(fecha), QUARTER(fecha), DATE_FORMAT(fecha, 'MMMM'), DATE_FORMAT(fecha, 'EEEE'), CASE WHEN DAYOFWEEK(fecha) IN (1, 7) THEN 'Fin de Semana' ELSE 'Día de Semana' END FROM (SELECT EXPLODE(SEQUENCE(TO_DATE('2022-01-01'), TO_DATE('2026-12-31'), INTERVAL 1 DAY)) AS fecha);
ALTER TABLE dim_fecha ADD CONSTRAINT pk_dim_fecha PRIMARY KEY(id_fecha) NOT ENFORCED;

### **Celda 3: Creación de Tablas de Hechos Optimizadas**

**Objetivo:** Aplicar la estrategia de optimización más importante a nuestras tablas de hechos, particionando por fecha y aplicando Z-Ordering en las claves foráneas.

**Código:**

In [0]:
%sql
-- **Tabla de Hechos de Pedidos**
CREATE OR REPLACE TABLE pedidos_silver (
  id_pedido BIGINT NOT NULL, id_usuario BIGINT, id_producto BIGINT, cantidad INT, monto_total DECIMAL(18, 2), ts_pedido TIMESTAMP, id_fecha DATE
)
PARTITIONED BY (id_fecha)
COMMENT 'Tabla de hechos de ventas, particionada por fecha para optimizar consultas de series de tiempo.';
INSERT INTO pedidos_silver
SELECT CAST(p.id_pedido AS BIGINT), CAST(p.id_usuario AS BIGINT), CAST(p.id_producto AS BIGINT), CAST(p.cantidad AS INT), CAST(p.monto AS DECIMAL(18, 2)),  CAST(p.fecha_pedido AS TIMESTAMP), TO_DATE(p.fecha_pedido)
FROM pedidos_bronze AS p WHERE p.id_pedido IS NOT NULL AND p.id_usuario IS NOT NULL AND p.id_producto IS NOT NULL;
OPTIMIZE pedidos_silver ZORDER BY (id_usuario, id_producto);
ALTER TABLE pedidos_silver ADD CONSTRAINT pk_pedidos PRIMARY KEY(id_pedido) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_usuarios FOREIGN KEY(id_usuario) REFERENCES usuarios_silver(id_usuario) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_productos FOREIGN KEY(id_producto) REFERENCES productos_silver(id_producto) NOT ENFORCED;
ALTER TABLE pedidos_silver ADD CONSTRAINT fk_pedidos_fecha FOREIGN KEY(id_fecha) REFERENCES dim_fecha(id_fecha) NOT ENFORCED;

-- **Tabla de Hechos de Devoluciones**
CREATE OR REPLACE TABLE devoluciones_silver (
  id_devolucion BIGINT NOT NULL, id_pedido BIGINT, motivo STRING, ts_devolucion TIMESTAMP, id_fecha DATE
)
PARTITIONED BY (id_fecha);
INSERT INTO devoluciones_silver
SELECT CAST(d.id_devolucion AS BIGINT), CAST(d.id_pedido AS BIGINT), d.motivo, CAST(d.fecha_devolucion AS TIMESTAMP), TO_DATE(d.fecha_devolucion)
FROM devoluciones_bronze AS d WHERE d.id_devolucion IS NOT NULL AND d.id_pedido IS NOT NULL;
OPTIMIZE devoluciones_silver ZORDER BY (id_pedido);
ALTER TABLE devoluciones_silver ADD CONSTRAINT pk_devoluciones PRIMARY KEY(id_devolucion) NOT ENFORCED;
ALTER TABLE devoluciones_silver ADD CONSTRAINT fk_devoluciones_pedidos FOREIGN KEY(id_pedido) REFERENCES pedidos_silver(id_pedido) NOT ENFORCED;
ALTER TABLE devoluciones_silver ADD CONSTRAINT fk_devoluciones_fecha FOREIGN KEY(id_fecha) REFERENCES dim_fecha(id_fecha) NOT ENFORCED;

  * **Explicación de la Optimización:** Particionar las tablas de hechos por `id_fecha` es la estrategia más efectiva, ya que la mayoría de las consultas analíticas filtran por rangos de tiempo. `ZORDER BY` en las claves foráneas (`id_usuario`, `id_producto`) co-localiza los datos relacionados, acelerando significativamente los `JOINs` con las dimensiones.